In [ ]:
#mount google drive so we can access our data files

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load dependencies
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf
import keras
from keras import backend as K

from tensorflow.keras.models import Model
from keras.layers import Activation, GlobalAveragePooling2D, Dropout, Dense, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential

from keras.applications import VGG16
from keras.applications import DenseNet121
from keras.applications import ResNet50

from keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import os
os.chdir('/content/drive/My Drive/')


In [ ]:
#load data from file
try:
    x_train = np.load("preprocessed_images/training.npy", mmap_mode=None, allow_pickle=False, fix_imports=True)
    x_validation = np.load("preprocessed_images/validation.npy",mmap_mode=None, allow_pickle=False, fix_imports=True)
    x_test = np.load("preprocessed_images/test.npy", mmap_mode = None,allow_pickle = False, fix_imports = True)
    print(x_train.shape)
    print(x_validation.shape)
    print(x_test.shape)
except:
    print("file not found")

(3112, 512, 512, 3)
(550, 512, 512, 3)
(1928, 512, 512, 3)


In [ ]:
#get labels for training and validation data

train_labels = pd.read_csv('train_dataframe.csv')
train_labels = train_labels['diagnosis']

validation_labels = pd.read_csv('validation_dataframe.csv')
validation_labels = validation_labels['diagnosis']

print(train_labels.shape[0])
print(validation_labels.shape[0])

3112
550


In [ ]:
#encode labels using ordinal encoding
#For example, if the original category is 2, our ordinal encoded label is [1,1,1,0,0] (encoded with 1's up to index 2)
def encode(labels):
    binary_labels = pd.get_dummies(labels).values
    cumulative_labels = np.empty(binary_labels.shape, dtype=binary_labels.dtype)
    cumulative_labels[:, 4] = binary_labels[:, 4]

    for i in range(3, -1, -1):
         cumulative_labels[:, i] = np.logical_or(binary_labels[:, i], cumulative_labels[:, i+1])
    return cumulative_labels

#encode labels
encoded_train_labels = encode(train_labels)
encoded_validation_labels = encode(validation_labels)

In [ ]:
#define AlexNet architecture
def AlexNet():
    model = Sequential(name='alexnet')
    model.add(Conv2D(96,(11,11),strides = (4,4), padding = 'valid', activation = 'relu', kernel_initializer = 'he_normal',input_shape = (512,512,3)))
    model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))
    model.add(Conv2D(256, kernel_size = (5,5), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))
    model.add(Conv2D(384, kernel_size = (3,3), padding = 'same', activation= 'relu'))
    model.add(Conv2D(384, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(Conv2D(256, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))
    model.add(Flatten())
    model.add(Dense(9216, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2048, activation = 'relu'))
    model.add(Dense(5, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.00001), metrics=['accuracy'])
    return model

In [ ]:
alex_net = AlexNet()
alex_net.summary()

Model: "alexnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 126, 126, 96)      34944     
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 62, 62, 96)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 62, 62, 256)       614656    
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 30, 30, 256)       0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 30, 30, 384)       885120    
                                                                 
 conv2d_23 (Conv2D)          (None, 30, 30, 384)       1327

In [ ]:
#train AlexNet model
alex_net = AlexNet()
history = alex_net.fit(x_train, encoded_train_labels, epochs = 10, batch_size = 8, verbose = 2, validation_data=(x_validation, encoded_validation_labels))

Epoch 1/10
389/389 - 15s - loss: 2.0071 - accuracy: 0.9553 - val_loss: 0.2849 - val_accuracy: 1.0000 - 15s/epoch - 40ms/step
Epoch 2/10
389/389 - 12s - loss: 0.3112 - accuracy: 0.9820 - val_loss: 0.2647 - val_accuracy: 0.9545 - 12s/epoch - 32ms/step
Epoch 3/10
389/389 - 12s - loss: 0.2760 - accuracy: 0.9839 - val_loss: 0.2604 - val_accuracy: 0.9818 - 12s/epoch - 32ms/step
Epoch 4/10
389/389 - 12s - loss: 0.2619 - accuracy: 0.9843 - val_loss: 0.2532 - val_accuracy: 1.0000 - 12s/epoch - 32ms/step
Epoch 5/10
389/389 - 12s - loss: 0.2557 - accuracy: 0.9830 - val_loss: 0.2510 - val_accuracy: 0.9982 - 12s/epoch - 32ms/step
Epoch 6/10
389/389 - 12s - loss: 0.2399 - accuracy: 0.9833 - val_loss: 0.2453 - val_accuracy: 1.0000 - 12s/epoch - 32ms/step
Epoch 7/10
389/389 - 12s - loss: 0.2279 - accuracy: 0.9836 - val_loss: 0.2409 - val_accuracy: 1.0000 - 12s/epoch - 32ms/step
Epoch 8/10
389/389 - 12s - loss: 0.2324 - accuracy: 0.9820 - val_loss: 0.2433 - val_accuracy: 1.0000 - 12s/epoch - 32ms/step


In [ ]:
#evaluate AlexNet
loss, acc = alex_net.evaluate(x_validation, encoded_validation_labels)
y_pred = alex_net.predict(x_validation)
print("Accuracy after training for 10 epochs: ", acc)

18/18 [==============================] - 0s 15ms/step
Accuracy after training for 10 epochs:  0.8218181729316711


In [ ]:
#add custom layers to pretrained architectures
def create_gap_layer():
    return GlobalAveragePooling2D()

def create_dropout_layer(value = 0.5):
    return Dropout(value)

def create_dense_layer():
    return Dense(5, activation='sigmoid')

global_average_pooling_layer = create_gap_layer()
dropout_layer = create_dropout_layer()
dense_layer = create_dense_layer()

In [ ]:
#define vgg16 architecture
def vgg_16(top_6 = False):
    vgg = VGG16(weights = 'imagenet', include_top = False, input_shape = (512,512,3))
    if not top_6:
        for layer in vgg.layers:
            layer.trainable = False
    else:
         for layer in vgg.layers[:13]:
            layer.trainable=False
    x = global_average_pooling_layer(vgg.layers[-1].output)
    x = dropout_layer(x)
    output = dense_layer(x)
    model = Model(vgg.layers[0].input,output)
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.00005), metrics=['accuracy'])
    return model

In [ ]:
vgg16 = vgg_16(True)
vgg16.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0   

In [ ]:
#perform horizontal flipping, vertical flipping, rotation, and zoom -- helps prevent overfitting due to small size of our dataset
generator = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=180,zoom_range = 0.2)
vgg16 = vgg_16(top_6 = True)
history = vgg16.fit(
                   generator.flow(x_train, encoded_train_labels, batch_size=12),
                    steps_per_epoch=len(x_train) / 12,
                    epochs=10,
                    initial_epoch=0,
                    verbose=1,
                    validation_data=(x_validation, encoded_validation_labels),
                    validation_steps=len(x_validation) / 12)

Epoch 1/10
259/259 [==============================] - 180s 686ms/step - loss: 0.2582 - accuracy: 0.9499 - val_loss: 0.1861 - val_accuracy: 0.9964
Epoch 2/10
259/259 [==============================] - 178s 686ms/step - loss: 0.1769 - accuracy: 0.8670 - val_loss: 0.1728 - val_accuracy: 0.6727
Epoch 3/10
259/259 [==============================] - 178s 687ms/step - loss: 0.1692 - accuracy: 0.7815 - val_loss: 0.1399 - val_accuracy: 0.8073
Epoch 4/10
259/259 [==============================] - 178s 684ms/step - loss: 0.1483 - accuracy: 0.7667 - val_loss: 0.1559 - val_accuracy: 0.8273
Epoch 5/10
259/259 [==============================] - 178s 685ms/step - loss: 0.1409 - accuracy: 0.7834 - val_loss: 0.1362 - val_accuracy: 0.8727
Epoch 6/10
259/259 [==============================] - 178s 688ms/step - loss: 0.1365 - accuracy: 0.7950 - val_loss: 0.1398 - val_accuracy: 0.7273
Epoch 7/10
259/259 [==============================] - 178s 686ms/step - loss: 0.1304 - accuracy: 0.7892 - val_loss: 0.1269 -

In [ ]:
#evaluate vgg16
test_loss, test_acc = vgg16.evaluate(x_validation,encoded_validation_labels)
y_pred = vgg16.predict(x_validation)
print("Test accuracy after training for 10 epochs: ", test_acc)

18/18 [==============================] - 2s 84ms/step
Test accuracy after training for 10 epochs:  0.918181836605072


In [ ]:
global_average_pooling_layer = create_gap_layer()
dropout_layer = create_dropout_layer()
dense_layer = create_dense_layer()

In [ ]:
#define model architecture for Densenet121 -- uses pretrained model on imagenet dataset
def DenseNet():
    densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(512,512,3))
    x = global_average_pooling_layer(densenet.layers[-1].output)
    x = dropout_layer(x)
    output = dense_layer(x)
    model = Model(densenet.layers[0].input,output)
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.00005), metrics=['accuracy'])
    return model

In [ ]:
densenet = DenseNet()
densenet.summary()

29084464/29084464 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 518, 518, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 256, 256, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  

In [ ]:
#train densenet
generator = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=180,zoom_range = 0.2)
densenet = DenseNet()
history = densenet.fit(
                    generator.flow(x_train, encoded_train_labels, batch_size=8),
                    steps_per_epoch=len(x_train) / 8,
                    epochs=10,
                    initial_epoch=0,
                    verbose=1,
                    validation_data=(x_validation, encoded_validation_labels),
                    validation_steps=len(x_validation) / 8)

Epoch 1/10
389/389 [==============================] - 236s 448ms/step - loss: 0.3043 - accuracy: 0.6099 - val_loss: 0.1464 - val_accuracy: 0.6855
Epoch 2/10
389/389 [==============================] - 171s 440ms/step - loss: 0.1750 - accuracy: 0.7085 - val_loss: 0.1200 - val_accuracy: 0.7164
Epoch 3/10
389/389 [==============================] - 173s 446ms/step - loss: 0.1450 - accuracy: 0.7561 - val_loss: 0.1279 - val_accuracy: 0.7036
Epoch 4/10
389/389 [==============================] - 173s 443ms/step - loss: 0.1349 - accuracy: 0.7381 - val_loss: 0.1144 - val_accuracy: 0.6473
Epoch 5/10
389/389 [==============================] - 173s 445ms/step - loss: 0.1369 - accuracy: 0.7683 - val_loss: 0.1241 - val_accuracy: 0.6782
Epoch 6/10
389/389 [==============================] - 172s 441ms/step - loss: 0.1245 - accuracy: 0.7792 - val_loss: 0.1206 - val_accuracy: 0.6582
Epoch 7/10
389/389 [==============================] - 173s 444ms/step - loss: 0.1195 - accuracy: 0.7792 - val_loss: 0.1104 -

In [ ]:
#evaluate densenet
test_loss, test_acc = densenet.evaluate(x_validation,encoded_validation_labels)
y_pred = densenet.predict(x_validation)
print("Accuracy after training for 10 epochs: ", test_acc)

18/18 [==============================] - 3s 86ms/step
Accuracy after training for 10 epochs:  0.6690909266471863


In [ ]:
global_average_pooling_layer = create_gap_layer()
dropout_layer = create_dropout_layer()
dense_layer = create_dense_layer()

In [ ]:
#define architecture for ResNet50 -- uses pretrained model on imagenet dataset
def ResNet_50():
    resnet = ResNet50(weights='imagenet',include_top=False,layers=keras.layers,input_shape=(512,512,3))
    x = global_average_pooling_layer(resnet.layers[-1].output)
    x = dropout_layer(x)
    output = dense_layer(x)
    model = Model(resnet.layers[0].input,output)
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.00005), metrics=['accuracy'])
    return model

In [ ]:
resnet = ResNet_50()
resnet.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 518, 518, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 256, 256, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 256, 256, 64)         256       ['conv1_conv[0][0]']          
 on)                         

In [ ]:
#train ResNet50
generator = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=180,zoom_range = 0.2)
history = resnet.fit(
                   generator.flow(x_train, encoded_train_labels, batch_size = 8),
                    steps_per_epoch=len(x_train) / 8,
                    epochs=10,
                    initial_epoch=0,
                    verbose=1,
                    validation_data=(x_validation, encoded_validation_labels),
                    validation_steps=len(x_validation) / 8)

Epoch 1/10
389/389 [==============================] - 214s 452ms/step - loss: 0.2246 - accuracy: 0.8191 - val_loss: 0.1684 - val_accuracy: 0.6709
Epoch 2/10
389/389 [==============================] - 175s 449ms/step - loss: 0.1594 - accuracy: 0.8262 - val_loss: 0.2022 - val_accuracy: 0.7182
Epoch 3/10
389/389 [==============================] - 173s 445ms/step - loss: 0.1382 - accuracy: 0.7706 - val_loss: 0.1125 - val_accuracy: 0.6691
Epoch 4/10
389/389 [==============================] - 176s 452ms/step - loss: 0.1269 - accuracy: 0.7956 - val_loss: 0.1247 - val_accuracy: 0.7545
Epoch 5/10
389/389 [==============================] - 176s 452ms/step - loss: 0.1168 - accuracy: 0.7918 - val_loss: 0.1145 - val_accuracy: 0.7636
Epoch 6/10
389/389 [==============================] - 176s 452ms/step - loss: 0.1094 - accuracy: 0.7773 - val_loss: 0.1092 - val_accuracy: 0.7018
Epoch 7/10
389/389 [==============================] - 178s 458ms/step - loss: 0.1068 - accuracy: 0.7905 - val_loss: 0.1774 -

In [ ]:
#evaluate ResNet50
loss, acc = resnet.evaluate(x_validation,encoded_validation_labels)
y_pred = resnet.predict(x_validation)
print("Accuracy after training for 10 epochs: ", acc)

18/18 [==============================] - 2s 74ms/step
Accuracy after training for 10 epochs:  0.7963636517524719
